In [1]:
!pip install pandas openpyxl

In [2]:
import pandas as pd
import sqlite3

In [3]:
file_path = 'C:/Users/anami/OneDrive/Desktop/Holiday_Comparisons.xlsx'
df_holidays = pd.read_excel("C:/Users/anami/OneDrive/Desktop/Holiday_Comparisons.xlsx", sheet_name='dataset')  
print("Dataset Sheet:")
display(df_holidays)

Dataset Sheet:


,TransactionID,Date,Time,CustomerID,Age,Gender,Location,StoreID,OnlineOrderFlag,ProductID,...,PaymentType,PromotionApplied,DiscountAmount,GiftWrap,ShippingMethod,DeliveryTime,Weather,Event,CustomerSatisfaction,ReturnFlag
0,1,2020-12-24,07:27:59,441,27,Other,City_15,NaN,True,106,...,Credit Card,False,0.000000,False,Standard,5.0,Snowy,NaN,5,False
1,2,2022-11-18,14:36:39,340,43,Male,City_13,NaN,True,816,...,Credit Card,True,0.000000,True,Express,3.0,Sunny,NaN,2,True
2,3,2019-12-26,20:23:50,31,25,Other,City_7,92.0,False,508,...,Credit Card,False,0.000000,False,NaN,NaN,Rainy,Christmas Market,4,False
3,4,2018-11-13,23:08:08,39,64,Male,City_20,100.0,False,710,...,Debit Card,False,0.000000,True,NaN,NaN,Rainy,NaN,1,True
4,5,2020-12-13,04:38:08,344,26,Other,City_10,90.0,False,687,...,Cash,False,0.000000,True,NaN,NaN,Sunny,Christmas Market,4,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,2022-12-21,12:05:06,245,25,Other,City_10,NaN,True,54,...,Credit Card,False,9.107212,True,Express,4.0,Sunny,Black Friday,4,False
9996,9997,2022-11-24,17:28:29,295,46,Female,City_17,NaN,True,324,...,Debit Card,True,4.153859,True,Express,5.0,Rainy,Black Friday,2,True
9997,9998,2021-11-02,23:20:58,202,46,Female,City_13,22.0,False,892,...,Cash,True,7.368328,True,NaN,NaN,Snowy,Black Friday,2,False
9998,9999,2020-11-11,11:33:17,310,64,Other,City_8,NaN,True,237,...,Online Payment,True,0.000000,True,Express,1.0,Sunny,Christmas Market,1,False


In [4]:
conn = sqlite3.connect(":memory:")
df_holidays.to_sql('dataset', conn, index=False, if_exists='replace')
print("Tables loaded successfully!")

Tables loaded successfully!


In [5]:
query = "SELECT Age, Category, ProductName FROM dataset;"
result = pd.read_sql_query(query, conn)
display(result)

,Age,Category,ProductName
0,27,Toys,Toys_Product
1,43,Clothing,Clothing_Product
2,25,Clothing,Clothing_Product
3,64,Toys,Toys_Product
4,26,Toys,Toys_Product
...,...,...,...
9995,25,Decorations,Decorations_Product
9996,46,Electronics,Electronics_Product
9997,46,Food,Food_Product
9998,64,Electronics,Electronics_Product


In [6]:
#1 Query to find type of purchases made by different age groups
query = """
SELECT 
    CASE
        WHEN Age BETWEEN 18 AND 24 THEN '18-24'
        WHEN Age BETWEEN 25 AND 34 THEN '25-34'
        WHEN Age BETWEEN 35 AND 44 THEN '35-44'
        WHEN Age BETWEEN 45 AND 54 THEN '45-54'
        WHEN Age BETWEEN 55 AND 64 THEN '55-64'
        WHEN Age >= 65 THEN '65+'
        ELSE 'Unknown'
    END AS AgeBracket,
    Category,
    COUNT(*) AS PurchaseCount
FROM dataset
GROUP BY AgeBracket, Category
ORDER BY AgeBracket, PurchaseCount DESC;
"""

result = pd.read_sql_query(query, conn)
display(result)


,AgeBracket,Category,PurchaseCount
0,18-24,Decorations,293
1,18-24,Toys,286
2,18-24,Electronics,283
3,18-24,Food,264
4,18-24,Clothing,247
5,25-34,Food,399
6,25-34,Decorations,390
7,25-34,Clothing,390
8,25-34,Toys,373
9,25-34,Electronics,368


In [7]:
# We notice that the category of Decorations & Toys ranks high for most age groups, whereas Clothing is not that popular

In [8]:
#2 Query to analyze which city has the highest demand, segregated on the basis of Event

query = """
SELECT Event, 
       Location, 
       StoreID, 
       Category,
       TotalPurchases
FROM (
    SELECT Event, 
           Location, 
           StoreID, 
           Category,
           COUNT(*) AS TotalPurchases,
           RANK() OVER (PARTITION BY Event ORDER BY COUNT(*) DESC) AS Rank
    FROM dataset
    GROUP BY Event, Location, StoreID, Category
) AS ranked_data
WHERE Rank = 1
ORDER BY Event;
"""

result = pd.read_sql_query(query, conn)
display(result)


,Event,Location,StoreID,Category,TotalPurchases
0,None,City_3,None,Decorations,25
1,Black Friday,City_11,None,Decorations,37
2,Christmas Market,City_13,None,Toys,25
3,Christmas Market,City_17,None,Clothing,25


In [9]:
# There is no concrete pattern noticed here, both events had different stores that noticed demand for different categories

In [10]:
#3 To check which mode of shopping observes higher demand and makes maximum sales
query = """
SELECT 
    CASE 
        WHEN OnlineOrderFlag = TRUE THEN 'Online'
        ELSE 'Offline'
    END AS SaleType,
    COUNT(*) AS TotalTransactions,
    SUM(TotalPrice) AS TotalRevenue,
    MAX(TotalPrice) AS MaxRevenue
FROM dataset
GROUP BY SaleType
ORDER BY TotalRevenue DESC;
"""

result = pd.read_sql_query(query, conn)
display(result)


,SaleType,TotalTransactions,TotalRevenue,MaxRevenue
0,Online,5418,900856.588123,499.982894
1,Offline,4582,753403.560117,499.458664


In [11]:
# Total revenue for Online sales is higher than offline, even though the maximum purchase for both are similar at ~$499

In [12]:
#4 Exploring the effect of promotional coupon on purchase pattern

query = """
SELECT 
    Category,
    SUM(CASE WHEN PromotionApplied = TRUE THEN 1 ELSE 0 END) AS PromoPurchases,
    SUM(CASE WHEN PromotionApplied = FALSE THEN 1 ELSE 0 END) AS NonPromoPurchases,
    COUNT(*) AS TotalPurchases
FROM dataset
GROUP BY Category
ORDER BY PromoPurchases DESC;
"""

result = pd.read_sql_query(query, conn)
display(result)


,Category,PromoPurchases,NonPromoPurchases,TotalPurchases
0,Toys,1047,964,2011
1,Electronics,1041,1012,2053
2,Decorations,1007,988,1995
3,Food,975,1016,1991
4,Clothing,971,979,1950


In [13]:
#Total Purchases are higher with promotional code applied than not, hence the idea of using a coupon motivates the customer to make the purchase

In [14]:
#5 Relationship between weather and purchase made

query = """
SELECT Weather, 
       COUNT(*) AS TotalPurchases
FROM dataset
WHERE OnlineOrderFlag = FALSE
GROUP BY Weather
ORDER BY TotalPurchases DESC;
"""

result = pd.read_sql_query(query, conn)
display(result)


,Weather,TotalPurchases
0,Rainy,1575
1,Snowy,1518
2,Sunny,1489


In [15]:
# No significant correlation found, but orders have increased in Rainy and Snowy weather 

In [16]:
#6 Are discounts a contributing factor towards Customer satisfaction?

query = """
SELECT 
    CASE 
        WHEN DiscountAmount BETWEEN 0 AND 10 THEN '0-10'
        WHEN DiscountAmount BETWEEN 11 AND 20 THEN '11-20'
        WHEN DiscountAmount BETWEEN 21 AND 30 THEN '21-30'
        WHEN DiscountAmount BETWEEN 31 AND 50 THEN '31-50'
        WHEN DiscountAmount > 50 THEN '50+'
        ELSE 'No Discount'
    END AS DiscountRange,
    AVG(CustomerSatisfaction) AS AvgSatisfaction,
    COUNT(*) AS TotalTransactions
FROM dataset
WHERE CustomerSatisfaction IS NOT NULL
GROUP BY DiscountRange
ORDER BY DiscountRange;
"""

result = pd.read_sql_query(query, conn)
display(result)


,DiscountRange,AvgSatisfaction,TotalTransactions
0,0-10,2.977584,7450
1,11-20,2.991628,1075
2,21-30,3.034743,662
3,31-50,2.966912,544
4,No Discount,2.985130,269


In [17]:
# A discount ranging from 21-30% depicts maximum satisfaction but this has no effect on the number of transactions made. 

In [18]:
#7 Which mode of payment is the most popular?

query = """
SELECT PaymentType, 
       COUNT(*) AS TotalTransactions
FROM dataset
GROUP BY PaymentType
ORDER BY TotalTransactions DESC;
"""

result = pd.read_sql_query(query, conn)
display(result)


,PaymentType,TotalTransactions
0,Cash,2540
1,Online Payment,2536
2,Credit Card,2497
3,Debit Card,2427


In [19]:
# All modes have transactions uniformly distributed

In [21]:
#8 Checking the maximum total revenue earned every year via the 2 holidays

query = """
SELECT 
    strftime('%Y', Date) AS Year,
    strftime('%m', Date) AS Month,
    Event,
    SUM(TotalPrice) AS TotalRevenue,
    COUNT(*) AS TotalTransactions
FROM dataset
WHERE Event IN ('Black Friday', 'Christmas') 
      AND strftime('%m', Date) IN ('11', '12')
GROUP BY Event, Year, Month
ORDER BY Event, Year, Month;
"""

result = pd.read_sql_query(query, conn)
display(result)



,Year,Month,Event,TotalRevenue,TotalTransactions
0,2018,11,Black Friday,66413.196069,407
1,2018,12,Black Friday,47007.742548,276
2,2019,11,Black Friday,63119.192147,380
3,2019,12,Black Friday,50156.018205,308
4,2020,11,Black Friday,66306.082664,420
5,2020,12,Black Friday,47200.914511,301
6,2021,11,Black Friday,70676.492355,433
7,2021,12,Black Friday,48772.362867,286
8,2022,11,Black Friday,67117.813615,423
9,2022,12,Black Friday,47779.261165,301


In [22]:
#9 Seasonality Analysis to determine best performing categories during each holiday

query = """
SELECT 
    Event,
    Category,
    SUM(Quantity) AS total_quantity
FROM dataset
GROUP BY Event, Category
ORDER BY Event, total_quantity DESC;
"""

result = pd.read_sql_query(query, conn)
display(result)


,Event,Category,total_quantity
0,None,Toys,1781
1,None,Decorations,1745
2,None,Food,1723
3,None,Clothing,1715
4,None,Electronics,1694
5,Black Friday,Electronics,2704
6,Black Friday,Food,2572
7,Black Friday,Toys,2500
8,Black Friday,Decorations,2469
9,Black Friday,Clothing,2457


In [23]:
# We find out that during Black Friday, the most popular categories are namely Electronics & Food, whereas during Christmas, Electronics & Toys have ranked on top. 

In [24]:
#8 When were the highest spent transactions made?

query = """
SELECT 
    CustomerID, 
    Event, 
    SUM(TotalPrice) AS Total_Spent
FROM dataset
GROUP BY CustomerID, Event
HAVING SUM(TotalPrice) > 500
ORDER BY Total_Spent DESC;
"""

result = pd.read_sql_query(query, conn)
display(result)


,CustomerID,Event,Total_Spent
0,90,Black Friday,3729.004259
1,481,Christmas Market,3618.386281
2,110,Black Friday,3152.690802
3,115,None,3139.463872
4,343,Black Friday,3107.451538
...,...,...,...
1285,171,Christmas Market,506.692709
1286,472,None,505.496371
1287,231,Christmas Market,504.620997
1288,120,Christmas Market,502.967116


In [25]:
#Black Friday saw the maximum sale made by 1 customer itself, followed by sales at the Christmas Market a few $100 behind.